In [172]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [88]:
data = pd.read_csv("data/KaggleV2-May-2016.csv")

In [89]:
data

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No
110523,3.596266e+12,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,1.557663e+13,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,9.213493e+13,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No


### Observations

In [90]:
len(data[data["Age"] == 0]),len(data[data["Age"] < 0]), len(data)

(3539, 1, 110527)

--> some irregularities in age : when unknown, =0?

### Data cleaning

##### Awaiting Days

In [91]:
data.ScheduledDay = pd.to_datetime(data.ScheduledDay)
data.AppointmentDay = pd.to_datetime(data.AppointmentDay)

In [92]:
data[data.ScheduledDay > data.AppointmentDay]
data['AppointmentDay'] = data['AppointmentDay'] + pd.Timedelta('1d') - pd.Timedelta('1s')
#pb for same day registration

In [93]:
data['AwaitingTime'] = (data.AppointmentDay - data.ScheduledDay).dt.days

##### Missed appointments

In [94]:
len(pd.unique(data.PatientId))

62299

In [95]:
len(data)

110527

In [96]:
mask = data.PatientId%1 != 0
data.loc[mask, "PatientId"] *= 100000

In [97]:
data.PatientId = data.PatientId - min(data.PatientId)

In [98]:
le = preprocessing.LabelEncoder()
trained_le = le.fit(data.PatientId)
data.PatientId = trained_le.transform(data.PatientId)

In [108]:
le = preprocessing.LabelEncoder()
trained_le = le.fit(data.AppointmentID)
data.AppointmentID = trained_le.transform(data.AppointmentID)

In [122]:
data = data.rename(columns={"No-show": "NoShow"});

In [123]:
data.loc[(data.NoShow == 'No'),'NoShow']= 0
data.loc[(data.NoShow == 'Yes'),'NoShow']= 1

In [124]:
cpt_previous = np.zeros(max(data.PatientId)+1)
cpt_previous_no_show = np.zeros(max(data.PatientId)+1)

In [125]:
data["NbPrevious"] = np.zeros(len(data))
data["NbPrevious_NoShow"] = np.zeros(len(data))

In [127]:
# number of previous appointments in May
for i in range(len(data.AppointmentID)):
    idx = data.PatientId[i]
    data["NbPrevious"][i] = cpt_previous[idx]
    data["NbPrevious_NoShow"][i] = cpt_previous_no_show[idx]
    cpt_previous[idx] +=1
    if data.NoShow[i]==1:
        cpt_previous_no_show[idx] +=1

<ipython-input-127-fc33df17c613>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["NbPrevious"][i] = cpt_previous[idx]
<ipython-input-127-fc33df17c613>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["NbPrevious_NoShow"][i] = cpt_previous_no_show[idx]


data.to_csv("data/df_new.csv", index= False)

# Modifs : proportion previous no show, dates

In [162]:
data = pd.read_csv("data/df_new.csv")

In [163]:
data["ProportionNoShow"] = data.NbPrevious_NoShow/data.NbPrevious

In [164]:
data

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,NoShow,AwaitingTime,NbPrevious,NbPrevious_NoShow,ProportionNoShow
0,30845,29332,F,2016-04-29 18:38:08+00:00,2016-04-29 23:59:59+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,0,0,0.0,0.0,NaN
1,55325,29079,M,2016-04-29 16:08:27+00:00,2016-04-29 23:59:59+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,0,0,0.0,0.0,NaN
2,15722,29115,F,2016-04-29 16:19:04+00:00,2016-04-29 23:59:59+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,0,0,0.0,0.0,NaN
3,9018,29299,F,2016-04-29 17:29:31+00:00,2016-04-29 23:59:59+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,0,0.0,0.0,NaN
4,23941,29073,F,2016-04-29 16:07:23+00:00,2016-04-29 23:59:59+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,0,0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,12800,35236,F,2016-05-03 09:15:35+00:00,2016-06-07 23:59:59+00:00,56,MARIA ORTIZ,0,0,0,0,0,1,0,35,1.0,1.0,1.000000
110523,14603,34127,F,2016-05-03 07:27:33+00:00,2016-06-07 23:59:59+00:00,51,MARIA ORTIZ,0,0,0,0,0,1,0,35,3.0,1.0,0.333333
110524,27287,22657,F,2016-04-27 16:03:52+00:00,2016-06-07 23:59:59+00:00,21,MARIA ORTIZ,0,0,0,0,0,1,0,41,0.0,0.0,NaN
110525,45960,22517,F,2016-04-27 15:09:23+00:00,2016-06-07 23:59:59+00:00,38,MARIA ORTIZ,0,0,0,0,0,1,0,41,1.0,0.0,0.000000


In [165]:
data = data.rename(columns={"ScheduledDay": "ScheduledTime"});
data["ScheduledTime"] = pd.to_datetime(data["ScheduledTime"])
data["ScheduledHour"] = data["ScheduledTime"].map(lambda x: x.hour)
data["ScheduledDay"] = data["ScheduledTime"].map(lambda x: x.day)
data["ScheduledMonth"] = data["ScheduledTime"].map(lambda x: x.month)
data["ScheduledYear"] = data["ScheduledTime"].map(lambda x: x.year)
data["ScheduledWeekDay"] = data["ScheduledTime"].map(lambda x: x.weekday())

In [166]:
data = data.rename(columns={"AppointmentDay": "AppointmentTime"});
data["AppointmentTime"] = pd.to_datetime(data["AppointmentTime"])
data["AppointmentHour"] = data["AppointmentTime"].map(lambda x: x.hour)
data["AppointmentDay"] = data["AppointmentTime"].map(lambda x: x.day)
data["AppointmentMonth"] = data["AppointmentTime"].map(lambda x: x.month)
data["AppointmentYear"] = data["AppointmentTime"].map(lambda x: x.year)
data["AppointmentWeekDay"] = data["AppointmentTime"].map(lambda x: x.weekday())

data.to_csv("data/df_new.csv", index= False)

### Modifs : Change appointment hour (Daniel random)

In [173]:
random.seed(42)
ind = pd.Index(np.arange(9,19), name = 'hour')
n = len(ind)
prob = pd.Series(np.random.normal(1/n, 0, n), index = ind, name = 'probability')

In [174]:
age  =  pd.Series([1, 0.98, 0.9, 0.725, 0.5, 0.275, 0.15, 0.0751, 0.02, 0], index = ind) / 1000
show = pd.Series([1, 0.8, 0.2, 0.08, 0, 0, 0.08, 0.2, 0.8, 1], index = ind) / 40
age  = age - age.mean()
show = show - show.mean()

In [175]:
for i in range(len(data)):
    p = prob + (data.loc[i, 'Age'] - 42) * age + (data.loc[i, 'NoShow'] - 0.2) * show
    data.loc[i, 'AppointmentHour'] = np.random.choice(ind, p = p)

In [176]:
data

,PatientId,AppointmentID,Gender,ScheduledTime,AppointmentTime,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,ScheduledHour,ScheduledDay,ScheduledMonth,ScheduledYear,ScheduledWeekDay,AppointmentHour,AppointmentDay,AppointmentMonth,AppointmentYear,AppointmentWeekDay
0,30845,29332,F,2016-04-29 18:38:08+00:00,2016-04-29 23:59:59+00:00,62,JARDIM DA PENHA,0,1,0,...,18,29,4,2016,4,13,29,4,2016,4
1,55325,29079,M,2016-04-29 16:08:27+00:00,2016-04-29 23:59:59+00:00,56,JARDIM DA PENHA,0,0,0,...,16,29,4,2016,4,13,29,4,2016,4
2,15722,29115,F,2016-04-29 16:19:04+00:00,2016-04-29 23:59:59+00:00,62,MATA DA PRAIA,0,0,0,...,16,29,4,2016,4,9,29,4,2016,4
3,9018,29299,F,2016-04-29 17:29:31+00:00,2016-04-29 23:59:59+00:00,8,PONTAL DE CAMBURI,0,0,0,...,17,29,4,2016,4,14,29,4,2016,4
4,23941,29073,F,2016-04-29 16:07:23+00:00,2016-04-29 23:59:59+00:00,56,JARDIM DA PENHA,0,1,1,...,16,29,4,2016,4,10,29,4,2016,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,12800,35236,F,2016-05-03 09:15:35+00:00,2016-06-07 23:59:59+00:00,56,MARIA ORTIZ,0,0,0,...,9,3,5,2016,1,14,7,6,2016,1
110523,14603,34127,F,2016-05-03 07:27:33+00:00,2016-06-07 23:59:59+00:00,51,MARIA ORTIZ,0,0,0,...,7,3,5,2016,1,12,7,6,2016,1
110524,27287,22657,F,2016-04-27 16:03:52+00:00,2016-06-07 23:59:59+00:00,21,MARIA ORTIZ,0,0,0,...,16,27,4,2016,2,18,7,6,2016,1
110525,45960,22517,F,2016-04-27 15:09:23+00:00,2016-06-07 23:59:59+00:00,38,MARIA ORTIZ,0,0,0,...,15,27,4,2016,2,12,7,6,2016,1
